In [1]:
import math
import numpy as np
import tensorflow as tf

import pandas as pd

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

def generate_cylinder_dataset(size):
  base_radius = np.random.uniform(low=0.5, high=2.0, size=size)
  height = np.random.uniform(low=0.5, high=2.0, size=size)
  volume = math.pi * (base_radius ** 2) * height
  return pd.DataFrame({'base_radius': base_radius, 'height': height, 'volume': volume})

cylinder_train_df = generate_cylinder_dataset(500000)
cylinder_test_df = generate_cylinder_dataset(1000)
print cylinder_train_df.head()

   base_radius    height     volume
0     1.685299  0.993687   8.866523
1     1.288739  1.569738   8.190435
2     1.174624  1.832504   7.943148
3     1.546657  1.976968  14.857219
4     0.700963  1.682245   2.596742


In [3]:
feature_columns = [
  tf.feature_column.numeric_column("base_radius"),
  tf.feature_column.numeric_column("height")
]

def make_input_fn(df, num_epochs, batch_size=512):
  return tf.estimator.inputs.pandas_input_fn(
    df,
    df['volume'],
    batch_size=batch_size,
    num_epochs=num_epochs,
    shuffle=True
  )


def predict_input_fn():
  features = {
    "base_radius": tf.constant([1.5]),
    "height": tf.constant([2.0])
  }
  return features

In [4]:
model = tf.estimator.DNNRegressor(
  feature_columns=feature_columns,
  hidden_units=[64, 32, 32, 16],
  optimizer=tf.train.AdagradOptimizer(learning_rate=0.05)
)

model.train(make_input_fn(cylinder_train_df, None, 1000), max_steps=5000)
predictions = model.predict(predict_input_fn)
print predictions.next()
print math.pi * (1.5 ** 2) * 2.0

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4af566af50>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmp1TgOgV', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp1TgOgV/model.ckpt.
INFO:tensorflow:loss = 75342.77,

In [5]:
def print_rmse(model, name, df):
  metrics = model.evaluate(input_fn = make_input_fn(cylinder_test_df, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))

print_rmse(model, 'test', cylinder_test_df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-19-17:18:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1TgOgV/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-19-17:18:04
INFO:tensorflow:Saving dict for global step 5000: average_loss = 0.004330344, global_step = 5000, loss = 2.165172
RMSE on test dataset = 0.0658053532243


Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License